In [88]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np

In [89]:
wisata = pd.read_json("dataset.json")
wisata.head()

,id,name,category,location,image,rating,description,coordinate
0,1,Kawah Putih,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-159888094008...,4.8,Danau kawah vulkanik dengan air berwarna putih...,"{'latitude': -7.1655, 'longitude': 107.3999}"
1,2,Tangkuban Perahu,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-158866821440...,4.7,Gunung berapi aktif dengan kawah besar berbent...,"{'latitude': -6.7708, 'longitude': 107.6005}"
2,3,Farmhouse Lembang,Wisata Keluarga,"Lembang, Bandung",https://images.unsplash.com/photo-160056675235...,4.6,Taman rekreasi bertema pedesaan Eropa dengan r...,"{'latitude': -6.8117, 'longitude': 107.6175}"
3,4,Saung Angklung Udjo,Budaya,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151628044061...,4.9,Pusat budaya Sunda yang menampilkan pertunjuka...,"{'latitude': -6.9175, 'longitude': 107.6191}"
4,5,Kampung Daun,Kuliner,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151724813546...,4.5,Restoran alam dengan konsep gazebo-gazebo di a...,"{'latitude': -6.8506, 'longitude': 107.6339}"


In [90]:
wisata[['latitude', 'longitude']] = pd.json_normalize(wisata['coordinate'])
wisata.head()

,id,name,category,location,image,rating,description,coordinate,latitude,longitude
0,1,Kawah Putih,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-159888094008...,4.8,Danau kawah vulkanik dengan air berwarna putih...,"{'latitude': -7.1655, 'longitude': 107.3999}",-7.1655,107.3999
1,2,Tangkuban Perahu,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-158866821440...,4.7,Gunung berapi aktif dengan kawah besar berbent...,"{'latitude': -6.7708, 'longitude': 107.6005}",-6.7708,107.6005
2,3,Farmhouse Lembang,Wisata Keluarga,"Lembang, Bandung",https://images.unsplash.com/photo-160056675235...,4.6,Taman rekreasi bertema pedesaan Eropa dengan r...,"{'latitude': -6.8117, 'longitude': 107.6175}",-6.8117,107.6175
3,4,Saung Angklung Udjo,Budaya,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151628044061...,4.9,Pusat budaya Sunda yang menampilkan pertunjuka...,"{'latitude': -6.9175, 'longitude': 107.6191}",-6.9175,107.6191
4,5,Kampung Daun,Kuliner,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151724813546...,4.5,Restoran alam dengan konsep gazebo-gazebo di a...,"{'latitude': -6.8506, 'longitude': 107.6339}",-6.8506,107.6339


In [91]:
wisata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           100 non-null    int64  
 1   name         100 non-null    object 
 2   category     100 non-null    object 
 3   location     100 non-null    object 
 4   image        100 non-null    object 
 5   rating       100 non-null    float64
 6   description  100 non-null    object 
 7   coordinate   100 non-null    object 
 8   latitude     100 non-null    float64
 9   longitude    100 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 7.9+ KB


In [92]:
wisata.describe()

,id,rating,latitude,longitude
count,100.000000,100.000000,100.000000,100.000000
mean,50.500000,4.565000,-6.927086,107.563906
std,29.011492,0.155294,0.278158,0.518100
min,1.000000,4.200000,-7.733300,106.546600
25%,25.750000,4.475000,-6.957775,107.302700
50%,50.500000,4.600000,-6.897050,107.617350
75%,75.250000,4.700000,-6.733975,107.666100
max,100.000000,4.900000,-6.556200,108.663400


In [93]:
wisata.isna().sum()

id             0
name           0
category       0
location       0
image          0
rating         0
description    0
coordinate     0
latitude       0
longitude      0
dtype: int64

In [94]:
wisata = wisata.drop(columns=["coordinate"])

In [95]:
print(f"Jumlah data duplikat : {wisata.duplicated().sum()}")

Jumlah data duplikat : 0


In [96]:
wisata["category"].unique()

array(['Alam', 'Wisata Keluarga', 'Budaya', 'Kuliner', 'Pantai',
       'Kebun Binatang', 'Taman Kota', 'Pegunungan', 'Taman Hiburan',
       'Taman Alam', 'Taman Botani', 'Pemandian Air Panas', 'Sejarah',
       'Danau'], dtype=object)

In [97]:
wisata.loc[wisata["category"] == "Taman Alam", "category"] = "Taman"
wisata.loc[wisata["category"] == "Taman Botani", "category"] = "Taman"

In [98]:
# Gabungkan fitur menjadi satu kolom teks
def combine_features(row):
    return f"{row['category']} {row['location']} {row['description']}"

wisata["combined_text"] = wisata.apply(combine_features, axis=1)

In [99]:
# Ambil stopwords Bahasa Indonesia
factory = StopWordRemoverFactory()
stopwords_id = factory.get_stop_words()

In [100]:
# Pakai di TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_id)
tfidf_matrix = vectorizer.fit_transform(wisata["combined_text"])

In [101]:
def rekomendasi(preferensi_nama=None, preferensi_kategori=None, df=None, tfidf_matrix=None, vectorizer=None, top_n=5):
    """
    preferensi_nama: list nama tempat favorit (misal: ["Kawah Putih", "Tangkuban Perahu"])
    preferensi_kategori: list kategori favorit (misal: ["Alam", "Pantai"])
    df: DataFrame wisata
    tfidf_matrix: TF-IDF matrix untuk df["combined_text"]
    vectorizer: objek TfidfVectorizer yang sudah difit
    top_n: jumlah rekomendasi yang ingin dikembalikan
    """
    
    preferensi_vektor = []
    
    # Ambil vektor dari nama tempat
    if preferensi_nama:
        idx_nama = df[df["name"].isin(preferensi_nama)].index.tolist()
        if idx_nama:
            # Konversi ke array dense dengan .toarray() sebelum menghitung mean
            vektor_nama = tfidf_matrix[idx_nama].toarray()
            preferensi_vektor.append(vektor_nama)
    
    # Ambil vektor dari input kategori
    if preferensi_kategori:
        gabung_kategori = " ".join(preferensi_kategori)
        vektor_kategori = vectorizer.transform([gabung_kategori]).toarray()  # Konversi ke array dense
        preferensi_vektor.append(vektor_kategori)
    
    if not preferensi_vektor:
        return "Masukkan minimal satu preferensi: nama tempat dan/atau kategori."
    
    # Gabungkan seluruh vektor dengan stack terlebih dahulu
    # preferensi_vektor adalah list array, setiap elemen berbentuk (n, features), di sini n bisa >1 untuk vektor nama
    # Kita gunakan vstack dan kemudian hitung rata-rata secara row-wise.
    combined_vectors = np.vstack(preferensi_vektor)
    user_profile = np.mean(combined_vectors, axis=0, keepdims=True)
    
    # Hitung cosine similarity antara user_profile dan seluruh tfidf_matrix (konversikan tfidf_matrix ke dense)
    cosine_scores = cosine_similarity(user_profile, tfidf_matrix.toarray()).flatten()
    
    # Urutkan indeks berdasarkan similarity, descending
    rekomendasi_idx = cosine_scores.argsort()[::-1]
    
    # Jika ada preferensi nama, hindari mengembalikan item yang sudah dipilih
    if preferensi_nama:
        idx_nama = df[df["name"].isin(preferensi_nama)].index.tolist()
        rekomendasi_idx = [i for i in rekomendasi_idx if i not in idx_nama]
    
    # Ambil top_n rekomendasi
    rekomendasi_items = df.iloc[rekomendasi_idx][:top_n][["name", "category", "location", "rating"]]
    
    return rekomendasi_items

In [102]:
preferensi_nama = ["Saung Angklung Udjo", "Tebing Keraton"]
preferensi_kategori = ["Alam", "pegunungan", "Kuliner"]

rekomendasi_hasil = rekomendasi(
    preferensi_nama=preferensi_nama,
    preferensi_kategori=preferensi_kategori,
    df=wisata, 
    tfidf_matrix=tfidf_matrix,
    vectorizer=vectorizer,
    top_n=5
)

print(rekomendasi_hasil)

                            name       category              location  rating
71  Kampung Wisata Sindangbarang         Budaya     Bogor, Jawa Barat     4.4
62                Gunung Ciremai     Pegunungan  Kuningan, Jawa Barat     4.7
4                   Kampung Daun        Kuliner   Bandung, Jawa Barat     4.5
25                   Warung Lela        Kuliner   Bandung, Jawa Barat     4.5
73                Dago Dreampark  Taman Hiburan   Bandung, Jawa Barat     4.5
